# Layers and models

In [2]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

import sys
sys.path.append('../../../')

import numpy as np
import pandas as pd

from molgraph.chemistry import MolecularGraphEncoder
from molgraph.chemistry import AtomFeaturizer  
from molgraph.chemistry import BondFeaturizer
from molgraph.chemistry import features

### Construct a `MolecularGraphEncoder`

In [40]:
atom_encoder = AtomFeaturizer([
    features.Symbol({'C', 'N', 'O'}, oov_size=1),
    features.Hybridization({'SP', 'SP2', 'SP3'}, oov_size=1),
    features.HydrogenDonor(),
    features.HydrogenAcceptor(),
    features.Hetero()
])

bond_encoder = BondFeaturizer([
    features.BondType({'SINGLE', 'DOUBLE', 'TRIPLE', 'AROMATIC'}),
    features.Rotatable()
])

mol_encoder = MolecularGraphEncoder(atom_encoder, bond_encoder)
mol_encoder

MolecularGraphEncoder(atom_encoder=AtomFeaturizer(features=[Symbol(allowable_set={'N', 'C', 'O'}, ordinal=False, oov_size=1), Hybridization(allowable_set={'SP3', 'SP', 'SP2'}, ordinal=False, oov_size=1), HydrogenDonor(), HydrogenAcceptor(), Hetero()], dtype='float32', ndim=11), bond_encoder=BondFeaturizer(features=[BondType(allowable_set={'DOUBLE', 'AROMATIC', 'TRIPLE', 'SINGLE'}, ordinal=False, oov_size=0), Rotatable()], dtype='float32', ndim=5), positional_encoding_dim=20, self_loops=False)

### Obtain dataset as `pd.DataFrame`

In [4]:
path = tf.keras.utils.get_file(
    fname='ESOL.csv',
    origin='http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/ESOL.csv',
)
df = pd.read_csv(path)
df.head(3)

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles
0,Amigdalin,-0.974,1,457.432,7,3,7,202.32,-0.77,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...
1,Fenfuram,-2.885,1,201.225,1,2,2,42.24,-3.30,Cc1occc1C(=O)Nc2ccccc2
2,citral,-2.579,1,152.237,0,0,4,17.07,-2.06,CC(C)=CCCC(C)=CC(=O)


### Obtain SMILES ("x"; string representation of molecules) and associated labels ("y")

In [5]:
x, y = df['smiles'].values, df['measured log solubility in mols per litre'].values

### Obtain `GraphTensor` ("x") from SMILES

In [6]:
x = mol_encoder(x)

print(x, end='\n\n')
print('node_feature shape:', x.node_feature.shape)
print('edge_dst shape:    ', x.edge_dst.shape)
print('edge_src shape:    ', x.edge_src.shape)
print('edge_feature shape:', x.edge_feature.shape)

GraphTensor(node_feature=<tf.RaggedTensor: shape=(1128, None, 11), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, edge_feature=<tf.RaggedTensor: shape=(1128, None, 5), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, positional_encoding=<tf.RaggedTensor: shape=(1128, None, 20), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, edge_dst=<tf.RaggedTensor: shape=(1128, None), dtype=int32, ragged_rank=1, row_splits_dtype=int32>, edge_src=<tf.RaggedTensor: shape=(1128, None), dtype=int32, ragged_rank=1, row_splits_dtype=int32>)

node_feature shape: (1128, None, 11)
edge_dst shape:     (1128, None)
edge_src shape:     (1128, None)
edge_feature shape: (1128, None, 5)


## Graph neural network (GNN) `layers`

In [7]:
from molgraph.layers import (
    GCNConv, 
    GCNIIConv, 
    GINConv, 
    GraphSageConv, 
    GATConv, 
    GatedGCNConv, 
    GraphTransformerConv, 
    GMMConv, 
    MPNNConv, 
    Readout
)

from molgraph.layers import ops as layer_ops

GATConv.__init__.__annotations__

{'units': typing.Union[int, NoneType],
 'use_edge_features': bool,
 'num_heads': int,
 'merge_mode': typing.Union[str, NoneType],
 'self_projection': bool,
 'batch_norm': bool,
 'residual': bool,
 'dropout': typing.Union[float, NoneType],
 'attention_activation': typing.Union[NoneType, str, typing.Callable[[tensorflow.python.framework.ops.Tensor], tensorflow.python.framework.ops.Tensor]],
 'activation': typing.Union[NoneType, str, typing.Callable[[tensorflow.python.framework.ops.Tensor], tensorflow.python.framework.ops.Tensor]],
 'use_bias': bool,
 'kernel_initializer': typing.Union[str, keras.initializers.initializers_v2.Initializer],
 'bias_initializer': typing.Union[str, keras.initializers.initializers_v2.Initializer],
 'kernel_regularizer': typing.Union[keras.regularizers.Regularizer, NoneType],
 'bias_regularizer': typing.Union[keras.regularizers.Regularizer, NoneType],
 'activity_regularizer': typing.Union[keras.regularizers.Regularizer, NoneType],
 'kernel_constraint': typing.Un

### Pass `GraphTensor` directly to `layer`

In [13]:
layer = GATConv(units=128, use_edge_features=True, num_heads=8, merge_mode='concat')

out1 = layer(x)                 # with nested ragged tensors
out2 = layer(x.merge())         # with nested tensors

print(out1)
print()
print(out2)

tf.reduce_all(out1.node_feature.flat_values == out2.node_feature).numpy()

GraphTensor(node_feature=<tf.RaggedTensor: shape=(1128, None, 128), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, edge_feature=<tf.RaggedTensor: shape=(1128, None, 128), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, positional_encoding=<tf.RaggedTensor: shape=(1128, None, 20), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, edge_dst=<tf.RaggedTensor: shape=(1128, None), dtype=int32, ragged_rank=1, row_splits_dtype=int32>, edge_src=<tf.RaggedTensor: shape=(1128, None), dtype=int32, ragged_rank=1, row_splits_dtype=int32>)

GraphTensor(node_feature=<tf.Tensor: shape=(14991, 128), dtype=float32>, edge_feature=<tf.Tensor: shape=(30856, 128), dtype=float32>, positional_encoding=<tf.Tensor: shape=(14991, 20), dtype=float32>, edge_dst=<tf.Tensor: shape=(30856,), dtype=int32>, edge_src=<tf.Tensor: shape=(30856,), dtype=int32>, graph_indicator=<tf.Tensor: shape=(14991,), dtype=int32>)


True

## Custom `layer`

`molgraph.layers.ops` (in this notebook `layer_ops`) supply various helper functions for implementing a GNN layer. Below e.g., `layer_ops.propagate_node_features` is used to aggregate information from source nodes to destination nodes.

Without `edge_feature`

In [17]:


class MyGCNConv(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
    
    def build(self, input_shape):
        self.kernel = self.add_weight(
            name='kernel', 
            shape=(input_shape[-1], self.units),
            dtype=tf.float32,
            trainable=True)
        self.built = True
    
    def call(self, graph_tensor):
        graph_tensor_orig = graph_tensor
        if isinstance(graph_tensor.node_feature, tf.RaggedTensor):
            graph_tensor = graph_tensor.merge()
            
        node_feature_transformed = tf.matmul(graph_tensor.node_feature, self.kernel)
        node_feature_aggregated = layer_ops.propagate_node_features(
            node_feature_transformed, 
            graph_tensor.edge_src, 
            graph_tensor.edge_dst, 
            mode='mean')
        return graph_tensor_orig.update({'node_feature': node_feature_aggregated})
    
my_layer = MyGCNConv(128)

my_layer(x)

GraphTensor(node_feature=<tf.RaggedTensor: shape=(1128, None, 128), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, edge_feature=<tf.RaggedTensor: shape=(1128, None, 5), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, positional_encoding=<tf.RaggedTensor: shape=(1128, None, 20), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, edge_dst=<tf.RaggedTensor: shape=(1128, None), dtype=int32, ragged_rank=1, row_splits_dtype=int32>, edge_src=<tf.RaggedTensor: shape=(1128, None), dtype=int32, ragged_rank=1, row_splits_dtype=int32>)

With `edge_feature`

In [18]:
class MyGCNConvWithGates(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
        self.built_from_tensor = False
    
    def build_from_tensor(self, graph_tensor):
        """Custom build method (not using the builtin `build`)"""
        
        self.kernel_node = self.add_weight(
            name='kernel_node', 
            shape=(graph_tensor.node_feature.shape[-1], self.units),
            dtype=tf.float32,
            trainable=True)
        
        self.kernel_edge = self.add_weight(
            name='kernel_edge', 
            shape=(graph_tensor.edge_feature.shape[-1], self.units),
            dtype=tf.float32,
            trainable=True)
        
        self.built_from_tensor = True
    
    def call(self, graph_tensor):
        
        graph_tensor_orig = graph_tensor
        if isinstance(graph_tensor.node_feature, tf.RaggedTensor):
            graph_tensor = graph_tensor.merge()
            
        if not self.built_from_tensor:
            self.build_from_tensor(graph_tensor)
            
        node_feature_transformed = tf.matmul(graph_tensor.node_feature, self.kernel_node)
        edge_feature_transformed = tf.matmul(graph_tensor.edge_feature, self.kernel_edge)
        
        gate = tf.nn.leaky_relu(edge_feature_transformed)
        gate = layer_ops.softmax_edge_weights(gate, graph_tensor.edge_dst)
        
        node_feature_aggregated = layer_ops.propagate_node_features(
            node_feature_transformed, 
            graph_tensor.edge_src, 
            graph_tensor.edge_dst, 
            edge_weight=gate, 
            mode='mean')
        
        return graph_tensor_orig.update({
            'node_feature': node_feature_aggregated,
            'edge_feature': edge_feature_transformed
        })
    
my_layer = MyGCNConvWithGates(128)

my_layer(x)

GraphTensor(node_feature=<tf.RaggedTensor: shape=(1128, None, 128), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, edge_feature=<tf.RaggedTensor: shape=(1128, None, 128), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, positional_encoding=<tf.RaggedTensor: shape=(1128, None, 20), dtype=float32, ragged_rank=1, row_splits_dtype=int32>, edge_dst=<tf.RaggedTensor: shape=(1128, None), dtype=int32, ragged_rank=1, row_splits_dtype=int32>, edge_src=<tf.RaggedTensor: shape=(1128, None), dtype=int32, ragged_rank=1, row_splits_dtype=int32>)

### Implement Keras models using GNN `layers`

Split data into train/test

In [22]:
random_indices = np.random.permutation(np.arange(x.shape[0]))

x_train = x[random_indices[:800]]
x_test = x[random_indices[800:]]

y_train = y[random_indices[:800]]
y_test = y[random_indices[800:]]

### Option 1: Keras Sequential API

In [23]:
import molgraph as mg

sequential_model = tf.keras.Sequential([
    tf.keras.layers.Input(type_spec=x_train.unspecific_spec),
    GCNConv(128),
    GCNConv(128),
    GCNConv(128),
    Readout(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1)
])

sequential_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gcn_conv (GCNConv)          (None, None, 128)         4864      
                                                                 
 gcn_conv_1 (GCNConv)        (None, None, 128)         33408     
                                                                 
 gcn_conv_2 (GCNConv)        (None, None, 128)         33408     
                                                                 
 segment_pooling_readout (Se  (None, 128)              0         
 gmentPoolingReadout)                                            
                                                                 
 dense_10 (Dense)            (None, 512)               66048     
                                                                 
 dense_11 (Dense)            (None, 1)                 513       
                                                        

In [26]:
sequential_model.compile('adam', 'mse', ['mae'])
sequential_model.fit(x_train, y_train, epochs=30, verbose=2)
mse, mae = sequential_model.evaluate(x_test, y_test)
print(f"{mse = :.3f}\n{mae = :.3f}")

Epoch 1/30
25/25 - 2s - loss: 0.9730 - mae: 0.7487 - 2s/epoch - 92ms/step
Epoch 2/30
25/25 - 0s - loss: 0.7365 - mae: 0.6299 - 130ms/epoch - 5ms/step
Epoch 3/30
25/25 - 0s - loss: 0.8885 - mae: 0.7109 - 128ms/epoch - 5ms/step
Epoch 4/30
25/25 - 0s - loss: 0.8271 - mae: 0.6752 - 125ms/epoch - 5ms/step
Epoch 5/30
25/25 - 0s - loss: 0.9101 - mae: 0.7191 - 123ms/epoch - 5ms/step
Epoch 6/30
25/25 - 0s - loss: 0.8767 - mae: 0.7117 - 123ms/epoch - 5ms/step
Epoch 7/30
25/25 - 0s - loss: 0.9032 - mae: 0.7313 - 127ms/epoch - 5ms/step
Epoch 8/30
25/25 - 0s - loss: 0.7146 - mae: 0.6390 - 130ms/epoch - 5ms/step
Epoch 9/30
25/25 - 0s - loss: 0.7188 - mae: 0.6373 - 125ms/epoch - 5ms/step
Epoch 10/30
25/25 - 0s - loss: 0.7070 - mae: 0.6392 - 125ms/epoch - 5ms/step
Epoch 11/30
25/25 - 0s - loss: 0.7260 - mae: 0.6365 - 124ms/epoch - 5ms/step
Epoch 12/30
25/25 - 0s - loss: 0.6799 - mae: 0.6171 - 123ms/epoch - 5ms/step
Epoch 13/30
25/25 - 0s - loss: 0.6146 - mae: 0.5803 - 125ms/epoch - 5ms/step
Epoch 14/3

### Option 2: Keras functional API

In [27]:
inputs = tf.keras.layers.Input(type_spec=x_train.merge().unspecific_spec)
x = GCNConv(128)(inputs)
x = GCNConv(128)(x)
x = GCNConv(128)(x)
x = Readout()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(1)(x)
functional_model = tf.keras.Model(inputs=inputs, outputs=x)
functional_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 11)]              0         
                                                                 
 gcn_conv_3 (GCNConv)        (None, 128)               4864      
                                                                 
 gcn_conv_4 (GCNConv)        (None, 128)               33408     
                                                                 
 gcn_conv_5 (GCNConv)        (None, 128)               33408     
                                                                 
 segment_pooling_readout_1 (  (None, 128)              0         
 SegmentPoolingReadout)                                          
                                                                 
 dense_12 (Dense)            (None, 512)               66048     
                                                             

In [28]:
functional_model.compile('adam', 'mse', ['mae'])
functional_model.fit(x_train, y_train, epochs=30, verbose=2)
mse, mae = functional_model.evaluate(x_test, y_test)
print(f"{mse = :.3f}\n{mae = :.3f}")

Epoch 1/30
25/25 - 2s - loss: 4.2945 - mae: 1.6155 - 2s/epoch - 95ms/step
Epoch 2/30
25/25 - 0s - loss: 2.8229 - mae: 1.3541 - 124ms/epoch - 5ms/step
Epoch 3/30
25/25 - 0s - loss: 2.5423 - mae: 1.2628 - 124ms/epoch - 5ms/step
Epoch 4/30
25/25 - 0s - loss: 2.2094 - mae: 1.1681 - 131ms/epoch - 5ms/step
Epoch 5/30
25/25 - 0s - loss: 2.2424 - mae: 1.1842 - 126ms/epoch - 5ms/step
Epoch 6/30
25/25 - 0s - loss: 2.1514 - mae: 1.1649 - 123ms/epoch - 5ms/step
Epoch 7/30
25/25 - 0s - loss: 2.1544 - mae: 1.1625 - 125ms/epoch - 5ms/step
Epoch 8/30
25/25 - 0s - loss: 2.0617 - mae: 1.1226 - 125ms/epoch - 5ms/step
Epoch 9/30
25/25 - 0s - loss: 1.9535 - mae: 1.0972 - 124ms/epoch - 5ms/step
Epoch 10/30
25/25 - 0s - loss: 1.8884 - mae: 1.0562 - 128ms/epoch - 5ms/step
Epoch 11/30
25/25 - 0s - loss: 1.7336 - mae: 1.0230 - 123ms/epoch - 5ms/step
Epoch 12/30
25/25 - 0s - loss: 1.8817 - mae: 1.0589 - 122ms/epoch - 5ms/step
Epoch 13/30
25/25 - 0s - loss: 2.1456 - mae: 1.1702 - 122ms/epoch - 5ms/step
Epoch 14/3

### Option 3: Keras subclassing

Creating a custom Keras model allow for more flexibility.

In [29]:
class MyModel(tf.keras.Model):
    def __init__(self, gnn_units=128, dense_units=512):
        super().__init__()
        self.gcn_1 = GCNConv(gnn_units)
        self.gcn_2 = GCNConv(gnn_units)
        self.gcn_3 = GCNConv(gnn_units)
        self.readout = Readout()
        self.dense_1 = tf.keras.layers.Dense(512, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(1)
    
    def call(self, inputs):
        x0 = inputs
        x1 = self.gcn_1(x0)
        x2 = self.gcn_2(x1)
        x3 = self.gcn_3(x2)
        x1 = self.readout(x1)
        x2 = self.readout(x2)
        x3 = self.readout(x3)
        x = tf.concat([x1, x2, x3], axis=1)
        x = self.dense_1(x)
        return self.dense_2(x)
        
        
my_model = MyModel()

In [33]:
class MyModel(tf.keras.Model):
    def __init__(self, gnn_units=128, dense_units=512):
        super().__init__()
        self.gcn_1 = GCNConv(gnn_units)
        self.gcn_2 = GCNConv(gnn_units)
        self.gcn_3 = GCNConv(gnn_units)
        self.readout = Readout()
        self.dense_1 = tf.keras.layers.Dense(512, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(1)
    
    def call(self, inputs):
        x0 = inputs
        x1 = self.gcn_1(x0)
        x2 = self.gcn_2(x1)
        x3 = self.gcn_3(x2)
        x1 = self.readout(x1)
        x2 = self.readout(x2)
        x3 = self.readout(x3)
        x = tf.concat([x1, x2, x3], axis=1)
        x = self.dense_1(x)
        return self.dense_2(x)
        
        
my_model = MyModel()

my_model(x_train) # build

my_model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gcn_conv_9 (GCNConv)        multiple                  4864      
                                                                 
 gcn_conv_10 (GCNConv)       multiple                  33408     
                                                                 
 gcn_conv_11 (GCNConv)       multiple                  33408     
                                                                 
 segment_pooling_readout_3 (  multiple                 0         
 SegmentPoolingReadout)                                          
                                                                 
 dense_16 (Dense)            multiple                  197120    
                                                                 
 dense_17 (Dense)            multiple                  513       
                                                        

In [35]:
my_model.compile('adam', 'mse', ['mae'])
my_model.fit(x_train, y_train, epochs=30, verbose=2)
mse, mae = my_model.evaluate(x_test, y_test)
print(f"{mse = :.3f}\n{mae = :.3f}")

Epoch 1/30
25/25 - 3s - loss: 2.8839 - mae: 1.3493 - 3s/epoch - 110ms/step
Epoch 2/30
25/25 - 0s - loss: 2.6842 - mae: 1.2859 - 134ms/epoch - 5ms/step
Epoch 3/30
25/25 - 0s - loss: 2.2023 - mae: 1.1636 - 135ms/epoch - 5ms/step
Epoch 4/30
25/25 - 0s - loss: 2.1967 - mae: 1.1720 - 136ms/epoch - 5ms/step
Epoch 5/30
25/25 - 0s - loss: 2.1682 - mae: 1.1579 - 136ms/epoch - 5ms/step
Epoch 6/30
25/25 - 0s - loss: 1.9332 - mae: 1.0919 - 135ms/epoch - 5ms/step
Epoch 7/30
25/25 - 0s - loss: 1.9765 - mae: 1.1035 - 134ms/epoch - 5ms/step
Epoch 8/30
25/25 - 0s - loss: 2.1477 - mae: 1.1577 - 135ms/epoch - 5ms/step
Epoch 9/30
25/25 - 0s - loss: 1.9732 - mae: 1.1083 - 134ms/epoch - 5ms/step
Epoch 10/30
25/25 - 0s - loss: 1.7515 - mae: 1.0429 - 134ms/epoch - 5ms/step
Epoch 11/30
25/25 - 0s - loss: 1.6578 - mae: 1.0007 - 135ms/epoch - 5ms/step
Epoch 12/30
25/25 - 0s - loss: 1.7114 - mae: 1.0160 - 131ms/epoch - 5ms/step
Epoch 13/30
25/25 - 0s - loss: 1.5957 - mae: 0.9855 - 134ms/epoch - 5ms/step
Epoch 14/

### utilize `tf.data.Dataset`

In [36]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_train = ds_train.shuffle(800).batch(32).map(lambda x, y: (x.merge(), y))

ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
ds_test = ds_test.batch(32).map(lambda x, y: (x.merge(), y))


In [37]:
sequential_model.compile('adam', 'mse', ['mae'])
sequential_model.fit(ds_train, epochs=30, verbose=2)
mse, mae = sequential_model.evaluate(x_test, y_test)
print(f"{mse = :.3f}\n{mae = :.3f}")

Epoch 1/30
25/25 - 2s - loss: 0.7368 - mae: 0.6331 - 2s/epoch - 69ms/step
Epoch 2/30
25/25 - 0s - loss: 0.5568 - mae: 0.5576 - 116ms/epoch - 5ms/step
Epoch 3/30
25/25 - 0s - loss: 0.4925 - mae: 0.5142 - 121ms/epoch - 5ms/step
Epoch 4/30
25/25 - 0s - loss: 0.4680 - mae: 0.5071 - 117ms/epoch - 5ms/step
Epoch 5/30
25/25 - 0s - loss: 0.5276 - mae: 0.5262 - 126ms/epoch - 5ms/step
Epoch 6/30
25/25 - 0s - loss: 0.4683 - mae: 0.5101 - 115ms/epoch - 5ms/step
Epoch 7/30
25/25 - 0s - loss: 0.7421 - mae: 0.6353 - 118ms/epoch - 5ms/step
Epoch 8/30
25/25 - 0s - loss: 0.5661 - mae: 0.5624 - 119ms/epoch - 5ms/step
Epoch 9/30
25/25 - 0s - loss: 0.6187 - mae: 0.5758 - 115ms/epoch - 5ms/step
Epoch 10/30
25/25 - 0s - loss: 0.4843 - mae: 0.5187 - 118ms/epoch - 5ms/step
Epoch 11/30
25/25 - 0s - loss: 0.4198 - mae: 0.4663 - 115ms/epoch - 5ms/step
Epoch 12/30
25/25 - 0s - loss: 0.5075 - mae: 0.5233 - 116ms/epoch - 5ms/step
Epoch 13/30
25/25 - 0s - loss: 0.4685 - mae: 0.5125 - 115ms/epoch - 5ms/step
Epoch 14/3

### Save and load model with `tf.saved_model`

In [38]:
import tempfile
import shutil

file = tempfile.NamedTemporaryFile()
filename = file.name
file.close()

tf.saved_model.save(sequential_model, filename)
loaded_model = tf.saved_model.load(filename)

print(loaded_model(x_train).shape)

shutil.rmtree(filename)

Function `_wrapped_model` contains input name(s) args_0 with unsupported characters which will be renamed to args_0_9 in the SavedModel.
Found untraced functions such as dense_layer_call_fn, dense_layer_call_and_return_conditional_losses, dense_1_layer_call_fn, dense_1_layer_call_and_return_conditional_losses, dense_layer_call_fn while saving (showing 5 of 8). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /tmp/tmpv6iggcj5/assets


/home/alex/Envs/dl/lib/python3.8/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:522: UserWarning: Encoding a StructuredValue with type molgraph.tensors.graph_tensor.GraphTensorSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
Assets written to: /tmp/tmpv6iggcj5/assets


(800, 1)


### Save and load model with Keras

In [39]:
import tempfile
import shutil

file = tempfile.NamedTemporaryFile()
filename = file.name
file.close()

sequential_model.save(filename)
loaded_model = tf.keras.models.load_model(filename)

loaded_model.fit(ds_train, epochs=1)

shutil.rmtree(filename)

Function `_wrapped_model` contains input name(s) args_0 with unsupported characters which will be renamed to args_0_9 in the SavedModel.
Found untraced functions such as dense_layer_call_fn, dense_layer_call_and_return_conditional_losses, dense_1_layer_call_fn, dense_1_layer_call_and_return_conditional_losses, dense_layer_call_fn while saving (showing 5 of 8). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /tmp/tmprs6l5asq/assets


/home/alex/Envs/dl/lib/python3.8/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:522: UserWarning: Encoding a StructuredValue with type molgraph.tensors.graph_tensor.GraphTensorSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
Assets written to: /tmp/tmprs6l5asq/assets


25/25 [==============================] - 2s 5ms/step - loss: 0.6140 - mae: 0.5990
